## Building a symbolic classifier

MDL based algorithms encode data according to a given codetable

When calling ``.fit``, we iteratively look for the codetable that compress
the training data the best

**When we are done with training our model, we can benefit from the refined codetable 
to make some predictions**

In [1]:
import pandas as pd
from skmine.itemsets import SLIM
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
class TransactionEncoder(MultiLabelBinarizer):  # pandas DataFrames are easier to read ;)
    def transform(self, X):
        _X = super().transform(X)
        return pd.DataFrame(data=_X, columns=self.classes_)

In [3]:
transactions = [ 
     ['bananas', 'milk'], 
     ['milk', 'bananas', 'cookies'], 
     ['cookies', 'butter', 'tea'], 
     ['tea'],  
     ['milk', 'bananas', 'tea'], 
]
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D 

,bananas,butter,cookies,milk,tea
0,1,0,0,1,0
1,1,0,1,1,0
2,0,1,1,0,1
3,0,0,0,0,1
4,1,0,0,1,1


In [4]:
slim = SLIM()
slim.fit(D)

,usage
"(bananas, milk)","[0, 1, 4]"
(tea),"[2, 3, 4]"
(cookies),"[1, 2]"
(butter),[2]


We keep this **codetable** in mind, as we will later use it **to interpret our predictions**

----------
### First "predictions" 

We define a new transactional dataset, and can call our ``decision_function`` on it. This will yield ``distances`` w.r.t the encoding scheme provided by our codetable

In [5]:
new_transactions = [ 
   ['bananas', 'milk'], 
   ['milk', 'sirup', 'cookies'], 
   ['butter', 'tea'], 
   ['tea'],  
   ['milk', 'bananas', 'tea'], 
]
new_D = te.transform(new_transactions)
new_D

/usr/local/Caskroom/miniconda/base/envs/scikit-mine/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:995: UserWarning: unknown class(es) ['sirup'] will be ignored
  .format(sorted(unknown, key=str)))


,bananas,butter,cookies,milk,tea
0,1,0,0,1,0
1,0,0,1,1,0
2,0,1,0,0,1
3,0,0,0,0,1
4,1,0,0,1,1


In [6]:
codes = slim.decision_function(new_D)

In [7]:
pd.DataFrame([pd.Series(new_transactions), codes], index=['transaction', 'distance']).T

,transaction,distance
0,"[bananas, milk]",-1.58496
1,"[milk, sirup, cookies]",-2.16992
2,"[butter, tea]",-1.16992
3,[tea],-1.58496
4,"[milk, bananas, tea]",-0.584962


---------------
### Built-in interpretations
Now we can interpret codes for the new data, directly by **looking at the codetable inferred from training data**

First observations

* Entry 1 has the highest distance w.r.t the encoding scheme.
  You can see it contains `milk`, `sirup` and `cookies`. From the codetable we see `milk` and `cookies` are not    grouped together, while `sirup` has never been seen
  

*  Entry 4 has the lowest distance. It contains `bananas` and `milk`, which are grouped together in the codetable and have high occurence in the training data.

--------------
### Shortest code wins !!
Next, we are going to use an ensemble of SLIM encoding schemes, and utilize them via a ``OneVsRest`` methodology, to perform **multi-class classification**.
The methodology is very simple

1. We clone our base estimator as many time as we need (one per class)
2. We fit every estimator on entries corresponding to its class in the input data
3. When calling ``.predict``, we actually call ``.decision_function`` and get distances from our decision boundaries for every class
4. The shorted code wins : we choose the class with the lowest distance for a given transaction

In [8]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

In [9]:
pipe = Pipeline([
    ('transaction_encoder', TransactionEncoder(sparse_output=False)),
    ('slim', SLIM()),
])

In [10]:
transactions = [
    ['milk', 'bananas'],
    ['tea', 'New York Times', 'El Pais'],
    ['New York Times'],
    ['El Pais', 'The Economist'],
    ['milk', 'tea'],
    ['croissant', 'tea'],
    ['croissant', 'chocolatine', 'milk'],
]
y = [
    'foodstore', 
    'newspaper', 
    'newspaper', 
    'newspaper', 
    'foodstore',
    'bakery',
    'bakery',
]

In [11]:
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D

,El Pais,New York Times,The Economist,bananas,chocolatine,croissant,milk,tea
0,0,0,0,1,0,0,1,0
1,1,1,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,1
5,0,0,0,0,0,1,0,1
6,0,0,0,0,1,1,1,0


In [12]:
ovr = OneVsRestClassifier(SLIM())

In [13]:
ovr.fit(D, y=y)
ovr.estimators_

In [14]:
pd.DataFrame(ovr.decision_function(D), columns=ovr.classes_)

,bakery,foodstore,newspaper
0,-inf,-1.584962,-inf
1,-1.584962,-1.584962,-2.0
2,-inf,-inf,-2.0
3,-inf,-inf,-1.0
4,-1.584962,-0.584962,-inf
5,-0.584962,-1.584962,-inf
6,-1.584962,-1.584962,-inf


In [15]:
ovr.predict(D)

array(['foodstore', 'foodstore', 'newspaper', 'newspaper', 'foodstore',
       'bakery', 'foodstore'], dtype='<U9')